### Transfer learning / Keras

In [3]:
from keras.optimizers import RMSprop

from keras.applications import ResNet50, VGG16, MobileNet
from keras.models import Model
from keras.layers import (
    Input, Dense, 
    Dropout, BatchNormalization,
    Flatten)

from albumentations import (
    Compose, ShiftScaleRotate
)

from h5imagegenerator import HDF5ImageGenerator

Using TensorFlow backend.


In [4]:
base = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3),
)

In [5]:
base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
class Head:
    @staticmethod
    def build(base, num_classes):
        head = base.output
        
        head = Flatten()(head)
        head = Dense(512, activation='relu')(head)
        head = Dense(256, activation='relu')(head)
        head = BatchNormalization()(head)
        head = Dropout(0.5)(head)
        
        head = Dense(num_classes, activation='softmax')(head)
        
        return head

In [8]:
head = Head.build(base, num_classes=2)

In [9]:
model = Model(inputs=base.input, outputs=head)

In [10]:
for layer in base.layers:
    layer.trainable = False

In [13]:
num_epochs = 25
batch_size = 32

In [16]:
aug = Compose([
    ShiftScaleRotate(
        shift_limit=0.02,
        scale_limit=0.04,
        rotate_limit=15,
    )
])

train_gen = HDF5ImageGenerator(
    src= 'train.h5',
    scaler=True,
    num_classes=2,
    batch_size=batch_size,
    augmenter=aug,
)

val_gen = HDF5ImageGenerator(
    src= 'val.h5',
    scaler=True,
    num_classes=2,
    batch_size=batch_size,
)

In [17]:
rms = RMSprop(lr=0.001, rho=0.9)

callbacks = []

model.compile(
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    optimizer=rms
)

h = model.fit_generator(
    train_gen,
    validation_data=val_gen,
    #workers=10,
    #use_multiprocessing=True,
    verbose=1,
    epochs=num_epochs
)

Epoch 1/25
13/13 [==============================] - 6s 475ms/step - loss: 0.8290 - accuracy: 0.6960 - val_loss: 14.3223 - val_accuracy: 0.5100
Epoch 2/25
13/13 [==============================] - 2s 146ms/step - loss: 0.3549 - accuracy: 0.8392 - val_loss: 1.1159 - val_accuracy: 0.7500
Epoch 3/25
13/13 [==============================] - 2s 149ms/step - loss: 0.1522 - accuracy: 0.9372 - val_loss: 0.0000e+00 - val_accuracy: 0.5100
Epoch 4/25
13/13 [==============================] - 2s 150ms/step - loss: 0.1475 - accuracy: 0.9497 - val_loss: 16.3655 - val_accuracy: 0.5100
Epoch 5/25
13/13 [==============================] - 2s 149ms/step - loss: 0.1062 - accuracy: 0.9673 - val_loss: 1.9187 - val_accuracy: 0.6900
Epoch 6/25
13/13 [==============================] - 2s 155ms/step - loss: 0.0618 - accuracy: 0.9774 - val_loss: 4.5293 - val_accuracy: 0.7400
Epoch 7/25
13/13 [==============================] - 2s 151ms/step - loss: 0.0752 - accuracy: 0.9724 - val_loss: 6.6695 - val_accuracy: 0.5100
